In [0]:
%restart_python
%pip install boto3
import boto3
import os
from botocore.exceptions import NoCredentialsError
import datetime
import sys
sys.path.insert(0, '/Workspace/Shared')
import etl_helpers 

tablename = "dimextensiontypes"
runcycleid = etl_helpers.start_run_cycle(f"{tablename}")
os.makedirs("/dbfs/foi/dataload", exist_ok=True)  # make sure directory exists

try:
    spark.sql(f"DROP TABLE IF EXISTS {tablename};")

    query = f"""
        CREATE TABLE {tablename}
        USING DELTA AS
            SELECT
                extensionreasonid AS extensiontypeid,
                extensiontype AS extensiontypename,
                NULL AS createdby,
                NULL AS createddate,
                NULL AS modifiedby,
                NULL AS modifieddate,
                CASE
                    WHEN a.isactive = TRUE THEN 'Y'
                ELSE
                    'N'
                END AS cactive,
                NULL AS cdelete,
                NULL AS cacttype,
                reason AS extensionreason,
                NULL AS extensionactdesc,
                NULL AS vcshowninsr,
                NULL AS clsthirdparty,
                NULL AS vcdaysallowed,
                defaultextendedduedays AS iextensiondays
                FROM foi_mod.extensionreasons a
                order by extensionreasonid;  
        """

    # print(query)

    df = spark.sql(query)
    df.show()

    etl_helpers.end_run_cycle(runcycleid, 't', f"{tablename}")
except NoCredentialsError:
    print("Credentials not available")
    etl_helpers.end_run_cycle(runcycleid, 'f', f"{tablename}", "Credentials not available")
except Exception as e:
    if (str(e) == "no changes for today"):
        print("here")
        etl_helpers.end_run_cycle(runcycleid, 't', f"{tablename}")
    else:
        print(f"An error occurred: {e}")    
        etl_helpers.end_run_cycle(runcycleid, 'f', f"{tablename}", f"An error occurred: {e}")